In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tarkkaanko/amazon")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\mique\.cache\kagglehub\datasets\tarkkaanko\amazon\versions\1
